# Cryptocurrency Trading Strategy Analysis
## Performance-Optimized Regime-Based Portfolio Management

This notebook provides a comprehensive analysis of the cryptocurrency trading strategy results. All performance metrics are loaded from the actual backtest engine to ensure data accuracy.

**Key Features:**
- Live data loading from backtest engine
- Real-time performance calculations
- Interactive visualizations
- Comprehensive regime analysis

**Analysis Period:** 2024-2025 (1 Year)
- **Initial Capital:** $100,000
- **Risk-Free Rate:** 3.95% (10Y Treasury as of Jan 2, 2024)
- **Methodology:** Industry standard calculations (252 trading days)

In [35]:
# Environment Setup and Library Imports
import warnings
warnings.filterwarnings('ignore')

# Core Data Science Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pickle
import glob
from pathlib import Path
import json

# Interactive Visualization Libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Interactive Widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Configuration
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

import plotly.io as pio
pio.templates.default = "plotly_white"

print("✅ Environment setup complete - ready for analysis!")

✅ Environment setup complete - ready for analysis!


In [36]:
# Backtest Results Loader
print("📊 LOADING BACKTEST RESULTS")
print("="*50)

def load_backtest_results():
    """Load backtest results from the engine"""
    try:
        print("🚀 Running backtest engine to generate results...")
        
        # Import and run the backtest engine
        from backtest_engine import RegimeBasedBacktester
        
        # Initialize backtester with exact same settings as standalone script
        backtester = RegimeBasedBacktester(
            start_date='2024-01-01',
            end_date='2025-01-01',
            initial_capital=100000,
            risk_free_rate=0.0395
        )
        
        # Disable EV filter to match standalone script
        backtester.trading_strategy.use_ev_filter = False
        
        # Run the backtest
        print("📊 Preparing data and running backtest...")
        backtester.prepare_data(force_refresh=False)
        results = backtester.run_backtest(rebalance_frequency='D')
        
        # Extract trade data
        trades_data = []
        if backtester.trading_strategy.trades:
            for trade in backtester.trading_strategy.trades:
                trades_data.append({
                    'symbol': trade.symbol,
                    'entry_price': trade.entry_price,
                    'exit_price': trade.exit_price,
                    'quantity': trade.quantity,
                    'entry_timestamp': trade.entry_timestamp,
                    'exit_timestamp': trade.exit_timestamp,
                    'regime_id': trade.regime_id,
                    'pnl': trade.pnl,
                    'pnl_pct': trade.pnl_pct,
                    'exit_reason': trade.exit_reason
                })
        trades_df = pd.DataFrame(trades_data)
        
        # Display results
        total_return = results.get('total_return', 0)
        sharpe_ratio = results.get('sharpe_ratio', 0)
        final_value = results.get('final_value', 0)
        
        print(f"\n=== BACKTEST RESULTS ===")
        print(f"✅ Total Return: {total_return:.2f}%")
        print(f"🎯 Sharpe Ratio: {sharpe_ratio:.3f}")
        print(f"💰 Final Value: ${final_value:,.0f}")
        print(f"💼 Total Trades: {len(trades_df)}")
        print(f"📊 All analysis cells will use this live data")
        
        return results, trades_df, backtester.regime_data, backtester.crypto_data, True
        
    except Exception as e:
        print(f"❌ Error loading results: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, False

# Load the data
backtest_results, trades_df, regime_data, crypto_data, success = load_backtest_results()

if success:
    print("\n✅ All data loaded successfully - ready for analysis!")
else:
    print("❌ Failed to load backtest data")

📊 LOADING BACKTEST RESULTS
🚀 Running backtest engine to generate results...
📊 Preparing data and running backtest...
🔧 PREPARING BACKTEST DATA
1. Collecting cryptocurrency price data...
📁 Loading cached crypto data...
2. Creating regime features...
🔧 Creating regime-optimized features...
📊 Processing 4 cryptocurrencies for regime features
   📈 Processing bitcoin (366 data points)
   📈 Processing ethereum (366 data points)
   📈 Processing cardano (366 data points)
   📈 Processing solana (366 data points)
✅ Created 47 features, 335 observations
3. Generating historical regime assignments...
📁 Loading cached regime data...
✅ Data preparation complete!

🚀 STARTING REGIME-BASED BACKTEST
📅 Period: 2024-01-01 to 2025-01-01
💰 Initial Capital: $100,000.00
🔄 Rebalance Frequency: D
📊 Total trading timestamps: 335 (D frequency)
📊 Original data points: 335
⏳ Progress: 0.3% (1/335)
🔄 Regime Change: None → 4 (CONSERVATIVE)
✅ Opened BTCUSD: 0.0603 @ $65791.00 (SL: $61578.48, TP: $75901.06)
✅ Opened ET

In [37]:
# Executive Summary
if success and backtest_results:
    # Extract performance metrics
    total_return = backtest_results.get('total_return', 0)
    sharpe_ratio = backtest_results.get('sharpe_ratio', 0)
    annualized_return = backtest_results.get('annualized_return', 0) * 100
    annualized_volatility = backtest_results.get('annualized_volatility', 0) * 100
    max_drawdown = backtest_results.get('max_drawdown', 0)
    win_rate = backtest_results.get('win_rate', 0)
    final_value = backtest_results.get('final_value', 0)
    total_trades = len(trades_df) if trades_df is not None else 0
    
    print("📊 EXECUTIVE SUMMARY")
    print("="*50)
    print()
    print("This project demonstrates a sophisticated quantitative trading strategy")
    print("leveraging machine learning for systematic cryptocurrency portfolio management.")
    print("The system employs PCA, K-means clustering, and Expected Value (EV) optimization")
    print("to identify market regimes and optimize trading decisions.")
    print()
    
    print("🎯 KEY PERFORMANCE METRICS:")
    print(f"   📈 Total Return:        {total_return:.2f}%")
    print(f"   🔺 Sharpe Ratio:        {sharpe_ratio:.3f}")
    print(f"   📊 Annualized Return:   {annualized_return:.2f}%")
    print(f"   📉 Annualized Vol:      {annualized_volatility:.2f}%")
    print(f"   ⚠️  Max Drawdown:       {max_drawdown:.2f}%")
    print(f"   ✅ Win Rate:           {win_rate:.1f}%")
    print(f"   💼 Total Trades:        {total_trades}")
    print(f"   💰 Final Value:         ${final_value:,.0f}")
    print()
    
    print("🛠️ TECHNICAL SKILLS DEMONSTRATED:")
    print("   • Machine Learning: PCA dimensionality reduction, K-means clustering")
    print("   • Portfolio Management: Adaptive position sizing, risk optimization")
    print("   • Expected Value Analysis: Data-driven allocation strategies")
    print("   • Risk Management: Systematic drawdown control and volatility management")
    print("   • Software Engineering: Object-oriented Python, modular architecture")
    
else:
    print("❌ Executive Summary Unavailable - Run backtest cell above to load results")

📊 EXECUTIVE SUMMARY

This project demonstrates a sophisticated quantitative trading strategy
leveraging machine learning for systematic cryptocurrency portfolio management.
The system employs PCA, K-means clustering, and Expected Value (EV) optimization
to identify market regimes and optimize trading decisions.

🎯 KEY PERFORMANCE METRICS:
   📈 Total Return:        54.35%
   🔺 Sharpe Ratio:        1.437
   📊 Annualized Return:   38.75%
   📉 Annualized Vol:      24.21%
   ⚠️  Max Drawdown:       13.27%
   ✅ Win Rate:           53.3%
   💼 Total Trades:        137
   💰 Final Value:         $154,347

🛠️ TECHNICAL SKILLS DEMONSTRATED:
   • Machine Learning: PCA dimensionality reduction, K-means clustering
   • Portfolio Management: Adaptive position sizing, risk optimization
   • Expected Value Analysis: Data-driven allocation strategies
   • Risk Management: Systematic drawdown control and volatility management
   • Software Engineering: Object-oriented Python, modular architecture


In [38]:
# Portfolio Performance Analysis
if success and backtest_results:
    print("📈 PORTFOLIO PERFORMANCE ANALYSIS")
    print("="*50)
    
    # Use actual equity curve from the backtest engine
    equity_curve = backtest_results.get('equity_curve')
    if equity_curve:
        df = pd.DataFrame(equity_curve)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Calculate metrics using actual engine results
        initial_capital = backtest_results.get('initial_capital', 100000)
        running_max = df['portfolio_value'].expanding().max()
        drawdown = (df['portfolio_value'] - running_max) / running_max * 100
        returns = df['portfolio_value'].pct_change().dropna() * 100
        
        # Create interactive dashboard
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Portfolio Equity Curve', 'Drawdown Analysis', 'Returns Distribution', 'Rolling Sharpe Ratio'),
            specs=[[{}, {}], [{}, {}]]
        )
        
        # Portfolio equity curve
        fig.add_trace(go.Scatter(
            x=df['timestamp'], 
            y=df['portfolio_value'],
            mode='lines',
            name='Strategy Portfolio Value',
            line=dict(color='blue', width=2),
            hovertemplate='<b>Date:</b> %{x}<br><b>Value:</b> $%{y:,.0f}<extra></extra>'
        ), row=1, col=1)
        
        fig.add_hline(y=initial_capital, line_dash="dash", line_color="gray", 
                     annotation_text="Initial Capital", row=1, col=1)
        
        # Drawdown analysis
        fig.add_trace(go.Scatter(
            x=df['timestamp'], 
            y=drawdown,
            mode='lines',
            fill='tonexty',
            name='Drawdown',
            line=dict(color='red', width=1),
            fillcolor='rgba(255,0,0,0.1)',
            hovertemplate='<b>Date:</b> %{x}<br><b>Drawdown:</b> %{y:.2f}%<extra></extra>'
        ), row=1, col=2)
        
        # Returns distribution
        fig.add_trace(go.Histogram(
            x=returns,
            nbinsx=30,
            name='Returns',
            marker_color='lightblue',
            opacity=0.7
        ), row=2, col=1)
        
        # Rolling Sharpe ratio
        if len(returns) > 30:
            rolling_mean = returns.rolling(30).mean()
            rolling_std = returns.rolling(30).std()
            risk_free_daily = backtest_results.get('risk_free_rate', 3.95) / 252
            rolling_sharpe = (rolling_mean - risk_free_daily) / rolling_std * np.sqrt(252)
            
            fig.add_trace(go.Scatter(
                x=df['timestamp'][30:], 
                y=rolling_sharpe[30:],
                mode='lines',
                name='30-Day Rolling Sharpe',
                line=dict(color='purple', width=2)
            ), row=2, col=2)
        
        fig.update_layout(
            height=600,
            title_text="Portfolio Performance Dashboard",
            title_x=0.5,
            showlegend=True
        )
        
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=2)
        fig.update_xaxes(title_text="Return (%)", row=2, col=1)
        fig.update_xaxes(title_text="Date", row=2, col=2)
        
        fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1)
        fig.update_yaxes(title_text="Drawdown (%)", row=1, col=2)
        fig.update_yaxes(title_text="Frequency", row=2, col=1)
        fig.update_yaxes(title_text="Rolling Sharpe", row=2, col=2)
        
        fig.show()
        
        # Performance summary with GitHub-compatible output
        actual_annualized_return = backtest_results.get('annualized_return', 0) * 100
        actual_annualized_volatility = backtest_results.get('annualized_volatility', 0) * 100
        actual_sharpe_ratio = backtest_results.get('sharpe_ratio', 0)
        period_return = backtest_results.get('total_return', 0)
        max_dd = backtest_results.get('max_drawdown', 0)
        
        print("\n📊 PROFESSIONAL PORTFOLIO PERFORMANCE SUMMARY")
        print("-" * 50)
        print(f"Total Return:         {period_return:.2f}%")
        print(f"Annualized Return:    {actual_annualized_return:.2f}%") 
        print(f"Annualized Volatility: {actual_annualized_volatility:.2f}%")
        print(f"Maximum Drawdown:     {max_dd:.2f}%")
        print()
        print(f"Sharpe Ratio:         {actual_sharpe_ratio:.3f}")
        print(f"Risk-Free Rate:       {backtest_results.get('risk_free_rate', 3.95):.2f}%")
        print(f"Data Source:          Live Backtest Engine Results")
        
else:
    print("❌ Portfolio Analysis Unavailable - Run backtest cell above to load performance data")

📈 PORTFOLIO PERFORMANCE ANALYSIS



📊 PROFESSIONAL PORTFOLIO PERFORMANCE SUMMARY
--------------------------------------------------
Total Return:         54.35%
Annualized Return:    38.75%
Annualized Volatility: 24.21%
Maximum Drawdown:     13.27%

Sharpe Ratio:         1.437
Risk-Free Rate:       0.04%
Data Source:          Live Backtest Engine Results


In [39]:
# Professional Trading Statistics
if success and backtest_results and trades_df is not None and len(trades_df) > 0:
    print("📊 TRADING STATISTICS")
    print("="*50)
    
    # Get equity curve data
    equity_curve = backtest_results.get('equity_curve')
    if equity_curve:
        df = pd.DataFrame(equity_curve)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Calculate daily returns
        returns = df['portfolio_value'].pct_change().dropna()
        daily_returns_pct = returns * 100
        
        # Calculate comprehensive trading statistics
        
        # 1. Basic Performance Metrics
        total_return = backtest_results.get('total_return', 0)
        annualized_return = backtest_results.get('annualized_return', 0) * 100
        annualized_volatility = backtest_results.get('annualized_volatility', 0) * 100
        sharpe_ratio = backtest_results.get('sharpe_ratio', 0)
        final_value = backtest_results.get('final_value', 0)
        initial_capital = backtest_results.get('initial_capital', 100000)
        
        # 2. Risk Metrics
        max_drawdown = backtest_results.get('max_drawdown', 0)
        running_max = df['portfolio_value'].expanding().max()
        drawdown_series = (df['portfolio_value'] - running_max) / running_max * 100
        
        # Drawdown duration analysis
        in_drawdown = drawdown_series < -0.01  # More than 0.01% drawdown
        drawdown_periods = []
        start_dd = None
        for i, is_dd in enumerate(in_drawdown):
            if is_dd and start_dd is None:
                start_dd = i
            elif not is_dd and start_dd is not None:
                drawdown_periods.append(i - start_dd)
                start_dd = None
        if start_dd is not None:  # Still in drawdown at end
            drawdown_periods.append(len(in_drawdown) - start_dd)
        
        avg_drawdown_duration = np.mean(drawdown_periods) if drawdown_periods else 0
        max_drawdown_duration = max(drawdown_periods) if drawdown_periods else 0
        
        # Value at Risk (VaR) and Expected Shortfall (ES)
        var_95 = np.percentile(daily_returns_pct, 5)  # 95% VaR (5th percentile)
        var_99 = np.percentile(daily_returns_pct, 1)  # 99% VaR (1st percentile)
        es_95 = daily_returns_pct[daily_returns_pct <= var_95].mean()  # Expected shortfall at 95%
        
        # 3. Trade Analysis Statistics
        trades_df['entry_timestamp'] = pd.to_datetime(trades_df['entry_timestamp'])
        trades_df['exit_timestamp'] = pd.to_datetime(trades_df['exit_timestamp'])
        trades_df['hold_days'] = (trades_df['exit_timestamp'] - trades_df['entry_timestamp']).dt.days
        
        # Win/Loss Analysis
        winning_trades = trades_df[trades_df['pnl'] > 0]
        losing_trades = trades_df[trades_df['pnl'] < 0]
        
        win_rate = len(winning_trades) / len(trades_df) * 100
        avg_win = winning_trades['pnl'].mean() if len(winning_trades) > 0 else 0
        avg_loss = losing_trades['pnl'].mean() if len(losing_trades) > 0 else 0
        largest_win = trades_df['pnl'].max()
        largest_loss = trades_df['pnl'].min()
        
        # Profit Factor
        total_gross_profit = winning_trades['pnl'].sum() if len(winning_trades) > 0 else 0
        total_gross_loss = abs(losing_trades['pnl'].sum()) if len(losing_trades) > 0 else 1e-6
        profit_factor = total_gross_profit / total_gross_loss if total_gross_loss > 0 else float('inf')
        
        # Expectancy and Kelly Criterion
        expectancy = (win_rate/100 * avg_win) + ((100-win_rate)/100 * avg_loss)
        if avg_loss != 0:
            win_loss_ratio = abs(avg_win / avg_loss)
            kelly_percent = (win_rate/100 - (100-win_rate)/100 / win_loss_ratio) * 100
        else:
            win_loss_ratio = float('inf')
            kelly_percent = win_rate/100 * 100
        
        # 4. Advanced Performance Metrics
        
        # Sortino Ratio (downside deviation)
        downside_returns = returns[returns < 0]
        downside_deviation = np.sqrt(np.mean(downside_returns**2)) * np.sqrt(252)
        risk_free_rate = backtest_results.get('risk_free_rate', 3.95) / 100
        sortino_ratio = (annualized_return/100 - risk_free_rate) / downside_deviation if downside_deviation > 0 else 0
        
        # Calmar Ratio (return/max drawdown)
        calmar_ratio = (annualized_return/100) / (abs(max_drawdown)/100) if max_drawdown != 0 else float('inf')
        
        # Information Ratio (if we had a benchmark, using 0 as benchmark)
        tracking_error = returns.std() * np.sqrt(252)
        information_ratio = (annualized_return/100) / tracking_error if tracking_error > 0 else 0
        
        # Maximum consecutive wins/losses
        trade_results = (trades_df['pnl'] > 0).astype(int)  # 1 for win, 0 for loss
        max_consecutive_wins = 0
        max_consecutive_losses = 0
        current_wins = 0
        current_losses = 0
        
        for result in trade_results:
            if result == 1:  # Win
                current_wins += 1
                current_losses = 0
                max_consecutive_wins = max(max_consecutive_wins, current_wins)
            else:  # Loss
                current_losses += 1
                current_wins = 0
                max_consecutive_losses = max(max_consecutive_losses, current_losses)
        
        # Trade frequency analysis
        total_days = (trades_df['exit_timestamp'].max() - trades_df['entry_timestamp'].min()).days
        trades_per_month = len(trades_df) / (total_days / 30.44) if total_days > 0 else 0
        avg_hold_time = trades_df['hold_days'].mean()
        
        # Display comprehensive statistics
        print("\n🎯 TRADING PERFORMANCE METRICS")
        print("-" * 50)
        print()
        
        print("📈 RETURN METRICS:")
        print(f"   Total Return:        {total_return:.2f}%")
        print(f"   Annualized Return:   {annualized_return:.2f}%")
        print(f"   Annualized Volatility: {annualized_volatility:.2f}%")
        print(f"   Sharpe Ratio:        {sharpe_ratio:.3f}")
        print(f"   Final Value:         ${final_value:,.0f}")
        print()
        
        print("⚠️ RISK METRICS:")
        print(f"   Maximum Drawdown:    {max_drawdown:.2f}%")
        print(f"   VaR (95%):          {var_95:.2f}%")
        print(f"   VaR (99%):          {var_99:.2f}%")
        print(f"   Expected Shortfall:  {es_95:.2f}%")
        print(f"   Avg DD Duration:     {avg_drawdown_duration:.1f} days")
        print()
        
        print("📊 ADVANCED RATIOS:")
        print(f"   Sortino Ratio:       {sortino_ratio:.3f}")
        print(f"   Calmar Ratio:        {calmar_ratio:.3f}")
        print(f"   Information Ratio:   {information_ratio:.3f}")
        print(f"   Profit Factor:       {profit_factor:.2f}")
        print(f"   Kelly %:             {kelly_percent:.1f}%")
        print()
        
        print("💼 TRADE ANALYSIS:")
        print(f"   Total Trades:        {len(trades_df)}")
        print(f"   Win Rate:            {win_rate:.1f}%")
        print(f"   Average Win:         ${avg_win:.0f}")
        print(f"   Average Loss:        ${avg_loss:.0f}")
        print(f"   Win/Loss Ratio:      {win_loss_ratio:.2f}")
        print(f"   Expectancy:          ${expectancy:.0f}")
        print(f"   Largest Win:         ${largest_win:.0f}")
        print(f"   Largest Loss:        ${largest_loss:.0f}")
        print()
        
        print("⏱️ TRADING BEHAVIOR:")
        print(f"   Avg Hold Time:       {avg_hold_time:.1f} days")
        print(f"   Trades/Month:        {trades_per_month:.1f}")
        print(f"   Max Consecutive Wins: {max_consecutive_wins}")
        print(f"   Max Consecutive Losses: {max_consecutive_losses}")
        print(f"   Total Trading Days:  {total_days}")
        print(f"   Total P&L:           ${(final_value - initial_capital):,.0f}")
        print()
        
        print("🎯 KEY PERFORMANCE INSIGHTS:")
        risk_adj_quality = "excellent" if sharpe_ratio > 1.5 else "good" if sharpe_ratio > 1.0 else "moderate" if sharpe_ratio > 0.5 else "poor"
        downside_quality = "strong" if sortino_ratio > 1.5 else "adequate" if sortino_ratio > 1.0 else "weak"
        profit_quality = "exceeds industry benchmark (>1.5)" if profit_factor > 1.5 else "meets minimum threshold (>1.0)" if profit_factor > 1.0 else "needs improvement (<1.0)"
        
        print(f"   • Risk-Adjusted Performance: Sharpe ratio of {sharpe_ratio:.3f} indicates {risk_adj_quality} risk-adjusted returns")
        print(f"   • Downside Protection: Sortino ratio of {sortino_ratio:.3f} shows {downside_quality} downside risk management")
        print(f"   • Trade Efficiency: Profit factor of {profit_factor:.2f} {profit_quality}")
        print(f"   • Position Sizing: Kelly criterion suggests optimal position size of {kelly_percent:.1f}% per trade")
        print(f"   • Strategy Consistency: Win rate of {win_rate:.1f}% with average expectancy of ${expectancy:.0f} per trade")
        print()
        
        # Create detailed regime performance table
        print("🎭 REGIME-BASED PERFORMANCE BREAKDOWN:")
        print("-" * 50)
        
        regime_stats = []
        for regime_id in sorted(trades_df['regime_id'].unique()):
            regime_trades = trades_df[trades_df['regime_id'] == regime_id]
            regime_wins = regime_trades[regime_trades['pnl'] > 0]
            
            win_rate_regime = len(regime_wins)/len(regime_trades)*100 if len(regime_trades) > 0 else 0
            total_pnl = regime_trades['pnl'].sum()
            avg_pnl = regime_trades['pnl'].mean()
            best_trade = regime_trades['pnl'].max()
            worst_trade = regime_trades['pnl'].min()
            
            print(f"Regime {regime_id}:")
            print(f"   Trades: {len(regime_trades):<3} | Win Rate: {win_rate_regime:5.1f}% | Total P&L: ${total_pnl:8.0f}")
            print(f"   Avg P&L: ${avg_pnl:6.0f} | Best: ${best_trade:6.0f} | Worst: ${worst_trade:7.0f}")
            print()
    
else:
    print("❌ Professional Statistics Unavailable - Run backtest cell above to load trade data")

📊 TRADING STATISTICS

🎯 TRADING PERFORMANCE METRICS
--------------------------------------------------

📈 RETURN METRICS:
   Total Return:        54.35%
   Annualized Return:   38.75%
   Annualized Volatility: 24.21%
   Sharpe Ratio:        1.437
   Final Value:         $154,347

⚠️ RISK METRICS:
   Maximum Drawdown:    13.27%
   VaR (95%):          -1.71%
   VaR (99%):          -3.74%
   Expected Shortfall:  -2.91%
   Avg DD Duration:     18.6 days

📊 ADVANCED RATIOS:
   Sortino Ratio:       1.856
   Calmar Ratio:        2.920
   Information Ratio:   1.600
   Profit Factor:       1.74
   Kelly %:             22.6%

💼 TRADE ANALYSIS:
   Total Trades:        137
   Win Rate:            53.3%
   Average Win:         $1738
   Average Loss:        $-1141
   Win/Loss Ratio:      1.52
   Expectancy:          $393
   Largest Win:         $11918
   Largest Loss:        $-4501

⏱️ TRADING BEHAVIOR:
   Avg Hold Time:       36.4 days
   Trades/Month:        12.5
   Max Consecutive Wins: 10
   Max

In [40]:
# Advanced Risk Analysis & Performance Visualization
if success and backtest_results and trades_df is not None and len(trades_df) > 0:
    print("📊 ADVANCED RISK ANALYSIS")
    print("="*50)
    
    # Get equity curve data
    equity_curve = backtest_results.get('equity_curve')
    if equity_curve:
        df = pd.DataFrame(equity_curve)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Calculate daily returns
        returns = df['portfolio_value'].pct_change().dropna()
        daily_returns_pct = returns * 100
        
        # Rolling risk metrics
        window = 30  # 30-day rolling window
        rolling_vol = returns.rolling(window).std() * np.sqrt(252) * 100
        rolling_sharpe = (returns.rolling(window).mean() * 252 - backtest_results.get('risk_free_rate', 3.95)/100) / (returns.rolling(window).std() * np.sqrt(252))
        
        # Drawdown analysis
        running_max = df['portfolio_value'].expanding().max()
        drawdown_pct = (df['portfolio_value'] - running_max) / running_max * 100
        underwater_curve = drawdown_pct.copy()
        
        # Create comprehensive risk dashboard
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                'Rolling 30-Day Volatility', 'Rolling 30-Day Sharpe Ratio',
                'Underwater Curve (Drawdown)', 'Daily Returns Distribution',
                'Monthly Returns Heatmap', 'Trade P&L Distribution'
            ),
            specs=[[{}, {}], [{}, {}], [{}, {}]],
            vertical_spacing=0.1
        )
        
        # 1. Rolling Volatility
        fig.add_trace(go.Scatter(
            x=df['timestamp'][window:], 
            y=rolling_vol[window:],
            mode='lines',
            name='30-Day Rolling Volatility',
            line=dict(color='orange', width=2),
            hovertemplate='<b>Date:</b> %{x}<br><b>Volatility:</b> %{y:.2f}%<extra></extra>'
        ), row=1, col=1)
        
        # Add volatility threshold lines
        fig.add_hline(y=rolling_vol[window:].mean(), line_dash="dash", line_color="gray", 
                     annotation_text="Mean Vol", row=1, col=1)
        
        # 2. Rolling Sharpe Ratio
        fig.add_trace(go.Scatter(
            x=df['timestamp'][window:], 
            y=rolling_sharpe[window:],
            mode='lines',
            name='30-Day Rolling Sharpe',
            line=dict(color='purple', width=2),
            hovertemplate='<b>Date:</b> %{x}<br><b>Sharpe:</b> %{y:.3f}<extra></extra>'
        ), row=1, col=2)
        
        # Add Sharpe benchmark lines
        fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Zero", row=1, col=2)
        fig.add_hline(y=1, line_dash="dash", line_color="green", annotation_text="Good (>1)", row=1, col=2)
        
        # 3. Underwater Curve (Drawdown)
        fig.add_trace(go.Scatter(
            x=df['timestamp'], 
            y=underwater_curve,
            mode='lines',
            fill='tonexty',
            name='Drawdown %',
            line=dict(color='red', width=1),
            fillcolor='rgba(255,0,0,0.2)',
            hovertemplate='<b>Date:</b> %{x}<br><b>Drawdown:</b> %{y:.2f}%<extra></extra>'
        ), row=2, col=1)
        
        # 4. Daily Returns Distribution
        fig.add_trace(go.Histogram(
            x=daily_returns_pct,
            nbinsx=40,
            name='Daily Returns',
            marker_color='lightblue',
            opacity=0.7,
            hovertemplate='<b>Return Range:</b> %{x:.2f}%<br><b>Frequency:</b> %{y}<extra></extra>'
        ), row=2, col=2)
        
        # Add normal distribution overlay for comparison
        mu, sigma = daily_returns_pct.mean(), daily_returns_pct.std()
        x_norm = np.linspace(daily_returns_pct.min(), daily_returns_pct.max(), 100)
        y_norm = ((1/(sigma * np.sqrt(2*np.pi))) * np.exp(-0.5*((x_norm - mu)/sigma)**2)) * len(daily_returns_pct) * (daily_returns_pct.max() - daily_returns_pct.min()) / 40
        
        fig.add_trace(go.Scatter(
            x=x_norm, 
            y=y_norm,
            mode='lines',
            name='Normal Distribution',
            line=dict(color='red', width=2, dash='dash')
        ), row=2, col=2)
        
        # 5. Monthly Returns Heatmap
        df['year'] = df['timestamp'].dt.year
        df['month'] = df['timestamp'].dt.month
        df['returns'] = df['portfolio_value'].pct_change() * 100
        
        monthly_returns = df.groupby(['year', 'month'])['returns'].sum().reset_index()
        monthly_pivot = monthly_returns.pivot(index='year', columns='month', values='returns').fillna(0)
        
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        fig.add_trace(go.Heatmap(
            z=monthly_pivot.values,
            x=[months[i-1] for i in monthly_pivot.columns],
            y=monthly_pivot.index,
            colorscale='RdYlGn',
            colorbar=dict(title="Return %"),
            hovertemplate='<b>%{y} %{x}:</b> %{z:.2f}%<extra></extra>'
        ), row=3, col=1)
        
        # 6. Trade P&L Distribution
        fig.add_trace(go.Histogram(
            x=trades_df['pnl'],
            nbinsx=30,
            name='Trade P&L',
            marker_color='green',
            opacity=0.7,
            hovertemplate='<b>P&L Range:</b> $%{x:.0f}<br><b>Frequency:</b> %{y}<extra></extra>'
        ), row=3, col=2)
        
        # Add vertical line at break-even
        fig.add_vline(x=0, line_dash="dash", line_color="black", annotation_text="Break Even", row=3, col=2)
        
        # Update layout
        fig.update_layout(
            height=1200,
            title_text="Advanced Risk Analysis Dashboard",
            title_x=0.5,
            showlegend=False
        )
        
        # Update axis labels
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=2)
        fig.update_xaxes(title_text="Date", row=2, col=1)
        fig.update_xaxes(title_text="Daily Return (%)", row=2, col=2)
        fig.update_xaxes(title_text="Month", row=3, col=1)
        fig.update_xaxes(title_text="Trade P&L ($)", row=3, col=2)
        
        fig.update_yaxes(title_text="Volatility (%)", row=1, col=1)
        fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=2)
        fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
        fig.update_yaxes(title_text="Frequency", row=2, col=2)
        fig.update_yaxes(title_text="Year", row=3, col=1)
        fig.update_yaxes(title_text="Frequency", row=3, col=2)
        
        fig.show()
        
        # Calculate additional risk metrics
        var_95 = np.percentile(daily_returns_pct, 5)
        var_99 = np.percentile(daily_returns_pct, 1)
        es_95 = daily_returns_pct[daily_returns_pct <= var_95].mean()
        
        # Tail risk analysis
        tail_ratio = abs(daily_returns_pct[daily_returns_pct < var_95].mean()) / daily_returns_pct[daily_returns_pct > -var_95].mean()
        
        # Skewness and Kurtosis
        from scipy import stats
        skewness = stats.skew(daily_returns_pct)
        kurtosis = stats.kurtosis(daily_returns_pct)
        
        # Maximum consecutive losing days
        losing_days = (daily_returns_pct < 0).astype(int)
        max_consecutive_losing_days = 0
        current_losing_streak = 0
        
        for day in losing_days:
            if day == 1:
                current_losing_streak += 1
                max_consecutive_losing_days = max(max_consecutive_losing_days, current_losing_streak)
            else:
                current_losing_streak = 0
        
        # Display advanced risk summary
        print("\n⚠️ ADVANCED RISK ASSESSMENT")
        print("-" * 50)
        print()
        
        print("📊 DISTRIBUTION ANALYSIS:")
        skew_desc = "(Right-skewed - more extreme positive returns)" if skewness > 0.5 else "(Left-skewed - more extreme negative returns)" if skewness < -0.5 else "(Approximately symmetric)"
        kurt_desc = "(Fat tails - higher extreme event probability)" if kurtosis > 2 else "(Thin tails - lower extreme event probability)" if kurtosis < -1 else "(Normal tail behavior)"
        tail_desc = "(Heavy downside tail risk)" if tail_ratio > 1.5 else "(Balanced tail risk)"
        
        print(f"   Skewness:                {skewness:.3f} {skew_desc}")
        print(f"   Kurtosis:                {kurtosis:.3f} {kurt_desc}")
        print(f"   Tail Ratio:              {tail_ratio:.2f} {tail_desc}")
        print(f"   Max Consecutive Loss Days: {max_consecutive_losing_days}")
        print()
        
        print("🚨 EXTREME RISK METRICS:")
        var95_risk = "(High risk)" if var_95 < -5 else "(Moderate risk)" if var_95 < -2 else "(Low risk)"
        var99_risk = "(Very high extreme risk)" if var_99 < -8 else "(High extreme risk)" if var_99 < -5 else "(Moderate extreme risk)"
        
        print(f"   95% VaR (Daily):         {var_95:.2f}% {var95_risk}")
        print(f"   99% VaR (Daily):         {var_99:.2f}% {var99_risk}")
        print(f"   Expected Shortfall (95%): {es_95:.2f}%")
        print(f"   Worst Single Day:        {daily_returns_pct.min():.2f}%")
        print(f"   Best Single Day:         {daily_returns_pct.max():.2f}%")
        print()
        
        print("💡 PROFESSIONAL RISK ASSESSMENT:")
        vol_quality = "high" if rolling_vol[window:].mean() > 25 else "moderate" if rolling_vol[window:].mean() > 15 else "low"
        consistency = "consistent" if rolling_sharpe[window:].std() < 0.5 else "variable"
        dist_normal = 'Non-normal return distribution requires careful risk management' if abs(skewness) > 0.5 or abs(kurtosis) > 2 else 'Return distribution shows normal characteristics'
        
        print(f"   • Volatility Profile: Average 30-day volatility of {rolling_vol[window:].mean():.2f}% indicates {vol_quality} volatility strategy")
        print(f"   • Consistency: Rolling Sharpe ratio ranges from {rolling_sharpe[window:].min():.2f} to {rolling_sharpe[window:].max():.2f}, showing {consistency} risk-adjusted performance")
        print(f"   • Drawdown Recovery: Maximum underwater period provides insight into strategy resilience during adverse conditions")
        print(f"   • Distribution Characteristics: {dist_normal}")
    
else:
    print("❌ Advanced Risk Analysis Unavailable - Run backtest cell above to load performance data")

📊 ADVANCED RISK ANALYSIS



⚠️ ADVANCED RISK ASSESSMENT
--------------------------------------------------

📊 DISTRIBUTION ANALYSIS:
   Skewness:                2.503 (Right-skewed - more extreme positive returns)
   Kurtosis:                21.522 (Fat tails - higher extreme event probability)
   Tail Ratio:              0.90 (Balanced tail risk)
   Max Consecutive Loss Days: 6

🚨 EXTREME RISK METRICS:
   95% VaR (Daily):         -1.71% (Low risk)
   99% VaR (Daily):         -3.74% (Moderate extreme risk)
   Expected Shortfall (95%): -2.91%
   Worst Single Day:        -5.61%
   Best Single Day:         13.84%

💡 PROFESSIONAL RISK ASSESSMENT:
   • Volatility Profile: Average 30-day volatility of 23.32% indicates moderate volatility strategy
   • Consistency: Rolling Sharpe ratio ranges from -6.34 to 9.30, showing variable risk-adjusted performance
   • Drawdown Recovery: Maximum underwater period provides insight into strategy resilience during adverse conditions
   • Distribution Characteristics: Non-normal ret